<a href="https://colab.research.google.com/github/maarjunk/House-Prices-Regression/blob/main/HouseXG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import xgboost as xg
import numpy as np
import pandas as pd


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder ,StandardScaler
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score , mean_squared_error

In [ ]:
df = pd.read_csv('train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
df1 = df.copy()

In [ ]:
df1.isnull().sum().sort_values(ascending=False)

PoolQC         1453
MiscFeature    1406
Alley          1369
Fence          1179
FireplaceQu     690
               ... 
ExterQual         0
Exterior2nd       0
Exterior1st       0
RoofMatl          0
SalePrice         0
Length: 81, dtype: int64

In [ ]:
df1 = df1.drop(['PoolQC','MiscFeature','Alley','Fence','Id'] ,axis=1)


In [ ]:
df2 = df1

In [ ]:
x = df2.drop(['SalePrice'],axis=1)
y = df2['SalePrice'].copy()

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
cat_columns = x_train.select_dtypes(include=['object']).columns
num_columns = x_train.select_dtypes(include=['number']).columns

ohe =OneHotEncoder(handle_unknown='ignore')
scaler = StandardScaler()
myx = x_train

cat_transform = Pipeline([('imp',SimpleImputer(strategy='most_frequent')),
                          ('ohe',ohe)])

preprocessor  = ColumnTransformer(transformers = [('cat',cat_transform,cat_columns),('num',scaler,num_columns)],remainder='passthrough')

In [ ]:
xgr = xg.XGBRegressor(objective = 'reg:squarederror',missing = -999)
pipe = Pipeline([('transform',preprocessor),
                 ('model',xgr)])
pipe.fit(x,y)

Pipeline(steps=[('transform',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  Pipeline(steps=[('imp',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'Hous...
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=-999,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=None, ...))])

In [ ]:
y_ = pipe.predict(x_test)

In [ ]:
print(f'r2:{r2_score(y_test,y_)}')
print(f'RMSE:{np.sqrt(mean_squared_error(y_test,y_))}')

r2:0.9994441032316842
RMSE:2064.92361856399


In [ ]:
#param_grid ={'model__max_depth' : [4,5,6],
#             'model__learning_rate' : [ 0.1 , 0.01 , 0.05 ],
#             'model__gamma' : [ 0 , 0.5 , 1 ],
#             'model__reg_lambda' : [ 0 , 0.5 , 1 ],
#             'model__n_estimators' : [ 100 , 200 , 500],
#             'model__subsample'    : [0.9, 0.5,0.1],
#}

#myGrid = GridSearchCV(estimator = pipe , param_grid = param_grid  , scoring='r2' , cv = 3)

In [ ]:
#param_grid ={'model__max_depth' : [2,3,4],
#             'model__learning_rate' : [ 0.05 ],
#             'model__gamma' : [ 0 ],
#             'model__reg_lambda' : [ 0 ],
#             'model__n_estimators' : [ 500 , 750 , 1000 ],
#             'model__subsample'    : [0.9 , 1],
#}

#myGrid = GridSearchCV(estimator = pipe , param_grid = param_grid  , scoring='r2' , cv = 3)

In [ ]:
param_grid ={'model__max_depth' : [3],
             'model__learning_rate' : [ 0.05 ],
             'model__gamma' : [ 0 ],
             'model__reg_lambda' : [ 0 ],
             'model__n_estimators' : [ 1000 ],
             'model__subsample'    : [0.9],
}

myGrid = GridSearchCV(estimator = pipe , param_grid = param_grid  , scoring='r2' , cv = 3)

In [ ]:
myGrid.fit(x_train,y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('transform',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat',
                                                                         Pipeline(steps=[('imp',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('ohe',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', '...
                                                     max_leaves=None,
                                                     min_child_weight=None,
                                                     missing=-999,
                                                     monotone_constraints=None,
                                                     n_estimators=100,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     predictor=None,
                                                     random_state=None, ...))]),
             param_grid={'model__gamma': [0],
                         'model__learning_rate': [0.1, 0.25, 0.05],
                         'model__max_depth': [2, 3, 4],
                         'model__n_estimators': [1000],
                         'model__reg_lambda': [0], 'model__subsample': [0.9]},
             scoring='r2')

In [ ]:
myGrid.best_params_

{'model__gamma': 0,
 'model__learning_rate': 0.05,
 'model__max_depth': 3,
 'model__n_estimators': 1000,
 'model__reg_lambda': 0,
 'model__subsample': 0.9}

In [ ]:
bestXG1  = myGrid.best_estimator_

In [ ]:
test_pred = bestXG1.predict(x_test)
print(f'r2:{r2_score(y_test,test_pred)}')
print(f'RMSE:{np.sqrt(mean_squared_error(y_test,test_pred))}')

r2:0.918578140305043
RMSE:24990.642733954268


In [ ]:
dft = pd.read_csv('test.csv')
dft.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [ ]:
dft1 = dft.drop(['PoolQC','MiscFeature','Alley','Fence','Id'] ,axis=1)

In [ ]:
mypred = pipe.predict(dft1)

In [ ]:
GridPred = bestXG1.predict(dft1)

In [ ]:
myans = pd.DataFrame( GridPred , columns=['SalePrice'])

In [ ]:
myans['Id'] = dft['Id']
myans.head()

,SalePrice,Id
0,123318.054688,1461
1,169326.468750,1462
2,181064.250000,1463
3,192407.234375,1464
4,191520.140625,1465


In [ ]:
myans.to_csv('GridXG_std1.csv',index=False)